In [ ]:
import pandas as pd
from sequential.seq2pat import Seq2Pat, Attribute

# 1. CSVデータの読み込み
# CSVの構成: 1行が1回の購買履歴 (user_id, item, timestamp, store)
csv_file = "./sorted_interaction_base_data.csv" # 用意したファイル名に合わせてください
df = pd.read_csv(csv_file)

# ---------------------------------------------------------
# 2. データのソート（重要）
# Seq2Patで正しくパターン認識するために、順序を整えます。
# 第1キー: user_id (ユーザーごとにまとめる)
# 第2キー: timestamp (時系列順にする)
# 第3キー: item (同一時刻の場合は、辞書順/50音順に並べる)
# ---------------------------------------------------------
df_sorted = df.sort_values(
    by=["user_id", "timestamp", "item"],
    ascending=[True, True, True]
)

# 3. ユーザーごとにリスト化（集約）
# groupbyを使って、ユーザーIDごとに各カラムをリストに変換します
grouped = df_sorted.groupby('user_id').agg({
    'item': list,
    'timestamp': list,
    'store': list
})

# 4. Seq2Pat用のデータ抽出
# そのまま .tolist() するだけで「リストのリスト」になります
sequences = grouped['item'].tolist()
timestamps = grouped['timestamp'].tolist()
stores = grouped['store'].tolist()

# --- ここまででデータ準備完了 ---
# 1. Seq2Patオブジェクトの作成
seq2pat = Seq2Pat(sequences=sequences)

# 2. 属性（Attribute）の作成と制約の追加
# タイムスタンプ属性を作成
time_attr = Attribute(values=timestamps)


# 例: 買い物と買い物の間隔が 10 以下であること
#seq2pat.add_constraint(time_attr.gap() <= 10)

#必要であれば「間隔が1以上（同じ時間の連投は除く）」なども可能です
seq2pat.add_constraint(0 <= time_attr.gap() <= 0)

# 3. パターン抽出の実行
# min_frequency: そのパターンが出現するユーザー数の最小値
patterns = seq2pat.get_patterns(min_frequency=5)

print(f"発見されたパターン数: {len(patterns)}")
if len(patterns) > 0:
    print("--- 上位パターン例 ---")
    for p in patterns[:5]:
        print(p)

In [ ]:
# 1. Seq2Patの初期化
seq2pat = Seq2Pat(sequences=sequences)

# 2. 属性（Attribute）の定義
# タイムスタンプを使って、「どれくらいの間隔で買ったか」を制約にします
time_attr = Attribute(values=timestamps)

# -----------------------------------------------------------------------
# 【重要】離脱予測・スイッチ分析のための制約設定
# -----------------------------------------------------------------------

# 制約A: 時間間隔 (Gap Constraint)
# 「数ヶ月ぶりの購入」ではなく、「習慣的な購買の流れ」を見たいので、
# アイテム間の間隔が「5単位以内（例: 短期間）」であるものに限定します。
seq2pat.add_constraint(time_attr.gap() <= 5)

# 3. パターンの抽出実行
# min_frequency: データ量が少ない場合は 2~3、多い場合はユーザー数の 1% 程度に設定
# ここではサンプルデータなので「2人以上に出現」するパターンを探します
patterns = seq2pat.get_patterns(min_frequency=2)

# -----------------------------------------------------------------------
# 4. 結果の整形と「ビール」関連パターンのフィルタリング
# -----------------------------------------------------------------------
print(f"全発見パターン数: {len(patterns)}")

# ビールを含むパターンだけを抽出して表示（これが離脱分析の鍵になります）
beer_patterns = [p for p in patterns if "ビール" in p]

print("\n--- ビールに関連する購買パターン (Top 10) ---")
# パターンは (sequence, frequency) のタプルで返ってきます
# frequency（出現数）が多い順にソートして表示
beer_patterns.sort(key=lambda x: x[-1], reverse=True)

print(beer_patterns)
for row in beer_patterns[:10]:
    # *pat で「最後の要素以外すべて」を受け取り、freq で「最後の要素」を受け取ります
    *pat, freq = row
    
    print(f"{freq}人のユーザー: {pat}")
# -----------------------------------------------------------------------
# 5. 特徴量化への応用イメージ
# 「ビール -> チューハイ」のようなスイッチパターンがあるか確認
# -----------------------------------------------------------------------
print("\n--- 要注意パターン（スイッチ/離脱の予兆）の探索例 ---")
switch_candidates = [
    p for p in beer_patterns 
    if len(p) >= 2         # 長さが2以上（連鎖している）
    and p[0] == "ビール"    # 最初がビールで...
]

if switch_candidates:
    for row in switch_candidates:
        *pat, freq = row
        print(f"スイッチの可能性あり ({freq}人): {pat}")
else:
    print("今回のサンプルデータでは明確なスイッチパターンは見つかりませんでした。")

In [ ]:
# --- テクニック: 「アイテム」+「店名」の合成データを作る ---
# これにより「セブンでビール -> ライフでビール」のような移動をパターンとして発見できます
sequences_with_store = []
for i in range(len(sequences)):
    # zipを使って、アイテムと店名を「@」で結合
    new_seq = [f"{item}@{store}" for item, store in zip(sequences[i], stores[i])]
    sequences_with_store.append(new_seq)

# 確認
print("合成シーケンス例:", sequences_with_store[0][:3])
# 出力例: ['ビール@イトーヨーカドー', 'コーラ@セブンイレブン', 'お茶@ローソン']

In [ ]:
# 分析用のSeq2Patインスタンス作成
# ここでは「店名込み」のデータを使います
seq2pat = Seq2Pat(sequences=sequences_with_store)

# 時間属性の作成
time_attr = Attribute(values=timestamps)

# ------------------------------------------------------------------
# シナリオA: 「ビールの購入間隔が空いてきた」人の検出
# ------------------------------------------------------------------
print("\n=== A. ビールの購入間隔増大（Gap >= 7）の検出 ===")

# 制約: アイテム間の時間が「7以上（例: 1週間以上）」空いている場所を探す
# ※普段頻繁に買う人にとって、このパターンが出ると離脱の危険信号です
seq2pat.add_constraint(time_attr.gap() >= 7)

# 抽出実行（頻度が少なくても、この挙動がある人を見つけたいのでmin_frequency=1でもOK）
long_gap_patterns = seq2pat.get_patterns(min_frequency=1)

# 「ビール」が含まれるロングギャップパターンのみ表示
found_count = 0
for row in long_gap_patterns:
    *pat, freq = row # 可変長受け取り
    # パターンの最初と最後がビール（同一カテゴリ）なのに間隔が広いものを探す
    # (店名がついてるので "ビール" という文字が含まれるかで判定)
    if "ビール" in pat[0] and "ビール" in pat[-1]:
        print(f"間隔増大アラート: {pat} (出現人数: {freq})")
        found_count += 1
        if found_count >= 5: break # 表示制限

# ------------------------------------------------------------------
# シナリオB: 「店が変わった」人の検出（Store Change）
# ------------------------------------------------------------------
print("\n=== B. ビールを買う店が変わった（Store Switch）パターンの検出 ===")

# Seq2Patを再初期化（制約をリセットするため）
seq2pat_store = Seq2Pat(sequences=sequences_with_store)

# 制約: 今度は逆に、間隔は「そこそこ短い（<= 10）」＝習慣的に買っている中で店が変わった
seq2pat_store.add_constraint(time_attr.gap() <= 10)

patterns_store = seq2pat_store.get_patterns(min_frequency=1)

found_count = 0
for row in patterns_store:
    *pat, freq = row
    
    # 2回連続購入のパターンでチェック
    if len(pat) == 2 and "ビール" in pat[0] and "ビール" in pat[1]:
        # 店名を抽出して比較（"@"の後ろが店名）
        store1 = pat[0].split("@")[1]
        store2 = pat[1].split("@")[1]
        
        if store1 != store2:
            print(f"店舗スイッチ検知: {store1} -> {store2} ({freq}人)")
            found_count += 1
            if found_count >= 5: break

# ------------------------------------------------------------------
# シナリオC: 「代替品（チューハイ等）へ移った」人の検出
# ------------------------------------------------------------------
print("\n=== C. 代替品への移行（Substitution）の検出 ===")

# 元のアイテム名のみのシーケンスを使用（店名は不要なため）
seq2pat_sub = Seq2Pat(sequences=sequences)
seq2pat_sub.add_constraint(time_attr.gap() <= 5) # 直近の変化を知りたい

patterns_sub = seq2pat_sub.get_patterns(min_frequency=1)

found_count = 0
for row in patterns_sub:
    *pat, freq = row
    
    if len(pat) >= 2:
        # パターンの「最初がビール」で「最後がビール以外」
        first_item = pat[0]
        last_item = pat[-1]
        
        target_substitutes = ["チューハイ", "ハイボール", "発泡酒"] # 監視対象
        
        if first_item == "ビール" and last_item in target_substitutes:
            print(f"代替品スイッチ検知: {pat} ({freq}人)")
            found_count += 1
            if found_count >= 5: break

In [ ]:
from sequential.pat2feat import Pat2Feat # 特徴量生成器
from sequential.dpm import dichotomic_pattern_mining, DichotomicAggregation # 2群比較マイニング

# 1. データの準備（例としてラベルを適当に生成）
# 実際には、ユーザーごとの離脱フラグ(0/1)を用意してください
import random
# sequences = [...] # 前のステップで作ったシーケンスデータ
# y = [...] # 0:継続, 1:離脱 のリスト (sequencesと同じ長さ)

# ここではデモ用にランダムにラベルを振ります
y = [random.choice([0, 1]) for _ in range(len(sequences))]

# 2. データを「継続群(Pos)」と「離脱群(Neg)」に分割
sequences_pos = [seq for seq, label in zip(sequences, y) if label == 0] # 継続
sequences_neg = [seq for seq, label in zip(sequences, y) if label == 1] # 離脱

print(f"継続ユーザー数: {len(sequences_pos)}")
print(f"離脱ユーザー数: {len(sequences_neg)}")

# 3. それぞれでSeq2Patモデルを作成
seq2pat_pos = Seq2Pat(sequences=sequences_pos)
seq2pat_neg = Seq2Pat(sequences=sequences_neg)

# ★ここがポイント！制約も入れられます
# 「直近の動き」を見るためにGap制約などをここに入れてもOKです
# seq2pat_neg.add_constraint(...) 

# 4. Dichotomic Pattern Mining (DPM) の実行
# 「離脱者(Neg)には頻出(min=2)するが、継続者(Pos)にはあまり出ない」パターンを探すなど
patterns = dichotomic_pattern_mining(
    seq2pat_pos, 
    seq2pat_neg,
    min_frequency_pos=4, # 継続者でも2人以上には出る
    min_frequency_neg=4  # 離脱者でも2人以上には出る
)

# ユニークなパターン（どちらか一方に偏っているパターンなど）を統合して取得
# DichotomicAggregation.union は「両方の群で見つかった特徴的なパターンの和集合」を取ります
dpm_patterns = patterns[DichotomicAggregation.union]

print(f"\n発見された識別パターン数: {len(dpm_patterns)}")
print("--- 離脱/継続を分けるパターンの例 ---")
for p in dpm_patterns[:5]:
    print(p)

# -----------------------------------------------------------
# 5. Pat2Featで機械学習用データへ一発変換
# -----------------------------------------------------------
# これが「上手い使い方」の肝です。
# 発見したパターンを使って、全データを0/1の特徴量行列に変換します。

pat2feat = Pat2Feat()

# 全データ（pos + neg）を結合して変換
all_sequences = sequences_pos + sequences_neg
all_labels = [0]*len(sequences_pos) + [1]*len(sequences_neg)

# get_featuresを実行すると、One-Hotエンコーディングされた疎行列(CSR matrix)が返ってきます
feature_matrix = pat2feat.get_features(
    sequences=all_sequences,
    patterns=dpm_patterns,
    drop_pattern_frequency=False # Trueにすると0/1、Falseだと出現回数になる
)
feature_matrix

                                  

In [10]:
# 1. 保存用の空リストを用意
selected_patterns = []

# 監視対象（ビールからの移行先）
target_substitutes = ["チューハイ", "ハイボール", "発泡酒"]

# --- ループ処理（ロジックは先ほどと同じ） ---
for row in patterns_sub:
    *pat, freq = row # 可変長対応
    
    # 条件: 長さが2以上、かつ「ビール -> 代替品」の動き
    if len(pat) >= 2:
        first_item = pat[0]
        last_item = pat[-1]
        
        if first_item == "ビール" and last_item in target_substitutes:
            # ★ここでprintする代わりにリストに追加します
            selected_patterns.append(pat)

print(f"特徴量として採用するパターン数: {len(selected_patterns)}")
print("採用パターン例:", selected_patterns[:3])

特徴量として採用するパターン数: 465
採用パターン例: [['ビール', 'チューハイ'], ['ビール', 'ナッツ', 'チューハイ'], ['ビール', '洗剤', '焼酎', 'チューハイ']]


In [13]:
import pandas as pd

# 特徴量生成器の準備
pat2feat = Pat2Feat()

# 2. 選んだパターンだけを使って特徴量を作成
# 注意: ここでは gap<=5 という「時間の制約」は無視され、
# 単に「その順番で買ったことがあるか」の判定になります。
feature_matrix_sub = pat2feat.get_features(
    sequences=sequences,         # 全ユーザーのデータ
    patterns=selected_patterns,  # さっき選んだ「ビール→チューハイ」等のパターン
    drop_pattern_frequency=True # 出現回数（またはTrueで0/1）
)

# 3. DataFrameに整形
# カラム名を「switch_パターン名」にしておくと分かりやすいです
col_names = [f"switch_{'_'.join(map(str, p))}" for p in selected_patterns]

df_switch_features = pd.DataFrame(
    feature_matrix_sub, # すでにDataFrameの場合は .values は不要
    columns=col_names
)

# ユーザーID（インデックス）を付与して確認
df_switch_features["user_id"] = range(1, len(sequences) + 1)

print("\n--- 作成されたスイッチ特徴量（先頭5行） ---")
print(df_switch_features.head())

/opt/miniconda3/envs/sony/lib/python3.12/site-packages/sequential/pat2feat.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['feature_' + str(i)] = df.apply(lambda row: is_satisfiable_in_rolling(row['sequence'], pattern,
/opt/miniconda3/envs/sony/lib/python3.12/site-packages/sequential/pat2feat.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['feature_' + str(i)] = df.apply(lambda row: is_satisfiable_in_rolling(row['sequence'], pattern,
/opt/miniconda3/envs/sony/lib/python3.12/site-packages/sequential/pat2feat.py:79


--- 作成されたスイッチ特徴量（先頭5行） ---
   switch_ビール_チューハイ  switch_ビール_ナッツ_チューハイ  switch_ビール_洗剤_焼酎_チューハイ  \
0               NaN                   NaN                     NaN   
1               NaN                   NaN                     NaN   
2               NaN                   NaN                     NaN   
3               NaN                   NaN                     NaN   
4               NaN                   NaN                     NaN   

   switch_ビール_焼酎_チューハイ  switch_ビール_紅茶_チューハイ  switch_ビール_紅茶_ナッツ_チューハイ  \
0                  NaN                  NaN                      NaN   
1                  NaN                  NaN                      NaN   
2                  NaN                  NaN                      NaN   
3                  NaN                  NaN                      NaN   
4                  NaN                  NaN                      NaN   

   switch_ビール_おにぎり_チューハイ  switch_ビール_おにぎり_ナッツ_チューハイ  \
0                    NaN                        NaN   
1             

/opt/miniconda3/envs/sony/lib/python3.12/site-packages/sequential/pat2feat.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['feature_' + str(i)] = df.apply(lambda row: is_satisfiable_in_rolling(row['sequence'], pattern,
/opt/miniconda3/envs/sony/lib/python3.12/site-packages/sequential/pat2feat.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['feature_' + str(i)] = df.apply(lambda row: is_satisfiable_in_rolling(row['sequence'], pattern,
